# EDA: Housing Data

In [72]:
#initial libraries needed for importing, cleaning, and initial visualization
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_style('darkgrid')

import warnings
warnings.filterwarnings("ignore")

# read the data.csv with pandas and assign it to the variable df
df = pd.read_csv('kc_house_data.csv')

# print the df info
print(df.info())

# check the df
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,0.0,0.0,...,8,1530,0.0,2009,0.0,98103,47.6993,-122.346,1530,1509
21593,6600060120,2/23/2015,400000.0,4,2.50,2310,5813,2.0,0.0,0.0,...,8,2310,0.0,2014,0.0,98146,47.5107,-122.362,1830,7200
21594,1523300141,6/23/2014,402101.0,2,0.75,1020,1350,2.0,0.0,0.0,...,7,1020,0.0,2009,0.0,98144,47.5944,-122.299,1020,2007
21595,291310100,1/16/2015,400000.0,3,2.50,1600,2388,2.0,NaN,0.0,...,8,1600,0.0,2004,0.0,98027,47.5345,-122.069,1410,1287


In [44]:
# find dups and missing values
duplicate_rows = df[df.duplicated()]
print(f"The df has {len(duplicate_rows)} duplicate rows.")

duplicate_id = df[df.duplicated('id')]
print(f'There are {len(duplicate_id)} duplicate values of id. ' 
      f'This represents {round(len(duplicate_id)/len(df),3)*100}% of all entries.')

for col in df:
    v = df[col].isna().sum()
    if v > 0:
        print(f'The column "{col}" has {v} missing values.')

The df has 0 duplicate rows.
There are 177 duplicate values of id. This represents 0.8% of all entries.
The column "waterfront" has 2376 missing values.
The column "view" has 63 missing values.
The column "yr_renovated" has 3842 missing values.


In [21]:
# count id dups with >1 unique dates
group = df.groupby(['id']).apply(lambda x: x['date'].nunique()).reset_index(name='count')    
groupcount = group['count'].loc[group['count'] >= 2]
print(f'There are {len(groupcount)} id values that have more than one unique sale date.')

There are 176 id values that have more than one unique sale date.


In [60]:
sold5 = df.yr_renovated.loc[lambda x: x >= 2015].count()
print(f'{sold5} properties have been renovated in the past 5 years.')
sold10 = df.yr_renovated.loc[lambda x: x >= 2010].count()
print(f'{sold10} properties have been renovated in the past 10 years.')
sold15 = df.yr_renovated.loc[lambda x: x >= 2005].count()
print(f'{sold15} properties have been renovated in the past 15 years.')
sold20 = df.yr_renovated.loc[lambda x: x >= 2000].count()
print(f'{sold20} properties have been renovated in the past 20 years.')
print(f'Only {round(sold20/len(df),2)*100}% of the total properties have been '
      'renovated in the last 20 years')

14 properties have been renovated in the past 5 years.
150 properties have been renovated in the past 10 years.
265 properties have been renovated in the past 15 years.
379 properties have been renovated in the past 20 years.
Only 2.0% of the total properties have been renovated in the last 20 years


In [69]:
age5 = df.yr_built.loc[lambda x: x >= 2015].count()
print(f'{age5} properties were built in the last 5 years.')
age20 = df.yr_built.loc[lambda x: x >= 2010].count()
print(f'{age20} properties were built in the last 20 years.')
age50 = df.yr_built.loc[lambda x: x >= 1970].count()
print(f'{age50} properties were built in the last 50 years.')
print(f'{round(age50/len(df),2)*100}% of the total properties were built '
      'in the last 50 years')

38 properties were built in the last 5 years.
1241 properties were built in the last 20 years.
11554 properties were built in the last 50 years.
53.0% of the total properties were built in the last 50 years


## initial observations:

All the properties in the dataset have sold in the market at least once and have selling dates ranging from 2014-2015. The duplicates in the column "id" have multiple sale dates. This means the same house was sold more than once. Since less than 1% of the properties have multiple sale dates and renovations in the last 20 years is below 2%, flipping houses in this market is likely difficult. And, over half of the properties sold were built in the last 50 years. This may show a preference for newer homes.